Download the text--Moby Dick, remove the gutenberg_ig column and remove rows that are empty

In [4]:
import re
#from functools import reduce
#import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()
importr('gutenbergr')

df = ro.r('gutenberg_download("2701")')
df.drop('gutenberg_id', inplace=True, axis=1)
df = df[df['text'] != ''].reset_index(drop=True)
#df.drop('index', inplace=True, axis=1)

AttributeError: module 'rpy2.situation' has no attribute 'get_rlib_path'

In [2]:
df.iloc[:15]['text']

NameError: name 'df' is not defined

Capture the title

In [ ]:
title = df.iloc[0]['text']
title

Capture the author

In [ ]:
author = df.iloc[1]['text']
author = re.sub(r'^By ','',author)
author

Remove the text up until Chapter 1 begins

In [ ]:
df.drop(df.index[0:df[df.text.str.contains('^CHAPTER 1\.')].index.tolist()[0]], inplace=True)
df.reset_index(inplace=True, drop=True)

Make a table of contents list with chapters and epilogue

In [ ]:
TOC = df[df.text.str.contains('^CHAPTER|Epilogue')]['text'].tolist()

In [ ]:
len(TOC)

Combine chapter data together

In [ ]:
corpus_dict = {}
i = 0
for k,v in df[1:].to_dict()['text'].items():
    if i not in corpus_dict:
        corpus_dict[i] = []
    if v not in TOC:
        corpus_dict[i].append(v)
    else:
        i += 1
        corpus_dict[i] = []

In [ ]:
len(corpus_dict)

Parse text into sentences, store them in a dict as prepartion to move into a dataframe with columns of title, chapter and sentence.

In [ ]:
from nltk.tokenize import sent_tokenize
sentence_dict = {}
sentence_dict['title'] = []
sentence_dict['chapter'] = []
sentence_dict['sentence'] = []
sentence_dict['author'] = []
for k,v in corpus_dict.items():
    for sentence in sent_tokenize(' '.join(v)):
        sentence_dict['title'].append(title)
        sentence_dict['chapter'].append(TOC[k])
        sentence_dict['sentence'].append(sentence)
        sentence_dict['author'].append(author)

Place the dict into a data frame so the data can be output to a csv easily.

In [ ]:
import pandas as pd
final_df = pd.DataFrame.from_dict(sentence_dict)

In [ ]:
final_df.sample(5)

In [ ]:
final_df.to_csv('./moby_dick.csv', index=False, encoding='utf-8')